Detection of DoS attacks for IoT network based on anomaly detection IDS technique and machine learning algorithms

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time

In [ ]:
path = "C:/Users/Esra/Desktop/IoT Network Intrusion Dataset/IoT Network Intrusion Dataset.csv"

df = pd.read_csv(path)

df.head()

In [ ]:
pip install ipykernel

In [ ]:
df['Cat'].value_counts()

In [ ]:
df.shape

In [ ]:
df= df[df['Cat'].isin(['DoS','Normal'])]
df.head()

In [ ]:
df['Cat'].value_counts()

In [ ]:
df.dtypes

DATA PREPROCESSING

Cleaning

In [ ]:
# Check Null Values
df.isnull().sum()

In [ ]:
check_nan = df.isnull().values.all()
check_nan

In [ ]:
#Finding categorical features
num_cols = df._get_numeric_data().columns

cate_cols = list(set(df.columns)-set(num_cols))
cate_cols.remove('Cat')
# cate_cols.remove('Attack Type')

cate_cols

In [ ]:

df.drop('Timestamp',axis = 1,inplace = True)
df.drop('Src_IP',axis = 1,inplace = True)
df.drop('Dst_IP',axis = 1,inplace = True)
df.drop('Sub_Cat',axis = 1,inplace = True)
df.drop('Label',axis = 1,inplace = True)
df.drop('Flow_ID',axis = 1,inplace = True)

Encoding 

In [ ]:
mydf = df.copy()
pmap = {'DoS':1,'Normal':0}
mydf['Cat'] = mydf['Cat'].map(pmap)
mydf.head()

In [ ]:

def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

mydf = clean_dataset(mydf)

In [ ]:
mydf.head()

In [ ]:
mydf.shape

In [ ]:
#Visualization
def bar_graph(feature):
    mydf[feature].value_counts().plot(kind="bar")

In [ ]:
bar_graph('Protocol')

In [ ]:
bar_graph('Cat')

In [ ]:
mydf.columns

In [ ]:
(mydf[[col for col in mydf if mydf[col].nunique() > 1]]).count()

In [ ]:
# df = df.dropna('columns')# drop columns with NaN

mydf = mydf[[col for col in mydf if mydf[col].nunique() > 1]]# keep columns where there are more than 1 unique values

corr = mydf.corr()

plt.figure(figsize=(15,12))

sns.heatmap(corr)

plt.show()

In [ ]:
df_std = df.std()
df_std = df_std.sort_values(ascending = True)
df_std

In [ ]:
mydf['Protocol'].value_counts()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score

In [ ]:

Y = mydf[['Cat']]
X = mydf.drop(['Cat',], axis=1)
sc = MinMaxScaler()
X = sc.fit_transform(X)
# Split test and train data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
print(X_train.shape, X_test.shape)
print(Y_train.shape, Y_test.shape)

Select features with CFS Algorithm

In [ ]:
import numpy as np
from skfeature.function.statistical_based import CFS

In [ ]:

feature_index = CFS.cfs(X_train, Y_train) # fit on the trainset




In [ ]:
#Order the features from high to low with CFS Algorithm
feature=[]
for i in feature_index:
  feature.append (mydf.columns[i])

In [ ]:
print(feature)

In [ ]:
feature_cfs_train=feature[:12]

In [ ]:
print(feature_cfs_train)

In [ ]:
#pip install sklearn-genetic

Get the important featured using genetic algorithm

In [ ]:
#pip uninstall numpy

In [ ]:
pip install  numpy==1.23.1

In [ ]:
from sklearn.datasets import load_breast_cancer
from genetic_selection import GeneticSelectionCV
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import numpy as np



X = mydf.drop(['Cat'], axis=1)
y = mydf[['Cat']].astype(float)
estimator = DecisionTreeClassifier()
model = GeneticSelectionCV(
    estimator, cv=5, verbose=0,
    scoring="accuracy", max_features=20,
    n_population=100, crossover_proba=0.5,
    mutation_proba=0.2, n_generations=50,
    crossover_independent_proba=0.5,
    mutation_independent_proba=0.04,
    tournament_size=3, n_gen_no_change=10,
    caching=True, n_jobs=-1)
model = model.fit(X, y)

# print the important features using the generic algorithm
print('Features:', X.columns[model.support_])

In [ ]:
print(X.columns[model.support_])
yii=X.columns[model.support_]
tt=yii.tolist()
print(tt)

Decision Tree

In [ ]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier

In [ ]:
model2 = DecisionTreeClassifier(criterion="entropy", max_depth = 4)
start_time = time.time()
#start to train the decision tree
model2.fit(X_train, Y_train.values.ravel())
end_time = time.time()

In [ ]:
print("Training time: ",end_time-start_time)

In [ ]:
start_time = time.time()
Y_test_pred2 = model2.predict(X_test)
end_time = time.time()
print("Testing time: ",end_time-start_time)


In [ ]:
print("Train score is:", model2.score(X_train, Y_train))
print("Test score is:",model2.score(X_test,Y_test))


In [ ]:
Y_pred = model2.predict(X_test)
Y_pred

using the GA algorithm

In [ ]:
x_train_GA=mydf[tt]
y_train_ga = mydf[['Cat']].astype(float)

using the CSF algorithm

In [ ]:
x_train_cfs=mydf[feature_cfs_train]
y_train_cfs = mydf[['Cat']].astype(float)

In [ ]:

arr = Y_test.to_numpy()
import numpy as np
result_tot = arr.flatten()


Train with GA features

In [ ]:
model2.fit(x_train_GA, y_train_ga.values.ravel())
print("Train score is:", model2.score(x_train_GA, y_train_ga))

Train with CFS algorithm

In [ ]:
model2.fit(x_train_cfs, y_train_cfs.values.ravel())
print("Train score is:", model2.score(x_train_cfs, y_train_cfs))

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
cm = confusion_matrix(Y_pred, result_tot)
ConfusionMatrixDisplay(cm, model2.classes_).plot()

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(Y_test, Y_test_pred2)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(Y_test, Y_test_pred2)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(Y_test, Y_test_pred2)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(Y_test, Y_test_pred2)
print('F1 score: %f' % f1)

RANDOM FOREST



In [ ]:
from sklearn.ensemble import RandomForestClassifier
model3 = RandomForestClassifier(n_estimators=30)
start_time = time.time()
model3.fit(X_train, Y_train.values.ravel())
end_time = time.time()
print("Training time: ",end_time-start_time)

In [ ]:
start_time = time.time()
Y_test_pred3 = model3.predict(X_test)
end_time = time.time()
print("Testing time: ",end_time-start_time)

Train with GA features

In [ ]:
model3.fit(x_train_GA, y_train_ga.values.ravel())
print("Train score is:", model3.score(x_train_GA, y_train_ga))

Train with CFS features

In [ ]:
model3.fit(x_train_cfs, y_train_cfs.values.ravel())
print("Train score is:", model3.score(x_train_cfs, y_train_cfs))

In [ ]:
cm = confusion_matrix(Y_test_pred3, result_tot)
ConfusionMatrixDisplay(cm, model3.classes_).plot()

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(Y_test, Y_test_pred3)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(Y_test, Y_test_pred3)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(Y_test, Y_test_pred3)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(Y_test, Y_test_pred3)
print('F1 score: %f' % f1)

In [ ]:
print("Train score is:", model3.score(x_train_cfs, y_train_cfs))
print("Test score is:",model3.score(x_train_cfs, y_train_cfs))

SUPPORT VECTOR MACHINE

In [ ]:
from sklearn.svm import SVC
model4 = SVC(gamma = 'scale')
start_time = time.time()
model4.fit(X_train, Y_train.values.ravel())
end_time = time.time()
print("Training time: ",end_time-start_time)

In [ ]:
start_time = time.time()
Y_test_pred4 = model4.predict(X_test)
end_time = time.time()
print("Testing time: ",end_time-start_time)

In [ ]:
print("Train score is:", model4.score(x_train_cfs, y_train_cfs))
print("Test score is:", model4.score(x_train_cfs, y_train_cfs))

Train with GA features

In [ ]:
model4.fit(x_train_GA, y_train_ga.values.ravel())
print("Train score is:", model4.score(x_train_GA, y_train_ga))

Train with CFA features

In [ ]:
model4.fit(x_train_cfs, y_train_cfs.values.ravel())
print("Train score is:", model4.score(x_train_cfs, y_train_cfs))

In [ ]:
cm = confusion_matrix(Y_test_pred4, result_tot)
ConfusionMatrixDisplay(cm, model4.classes_).plot()

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(Y_test, Y_test_pred4)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(Y_test, Y_test_pred4)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(Y_test, Y_test_pred4)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(Y_test, Y_test_pred4)
print('F1 score: %f' % f1)

KNN

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
model5 = KNeighborsRegressor(n_neighbors=3)
start_time = time.time()
model5.fit(X_train, Y_train.values.ravel())
end_time = time.time()
print("Training time: ",end_time-start_time)

In [ ]:
start_time = time.time()
Y_test_pred5 = model5.predict(X_test)
end_time = time.time()
print("Testing time: ",end_time-start_time)


In [ ]:
(Y_test_pred5)

In [ ]:
(result_tot)

In [ ]:
print("Train score is:", model5.score(X_train, Y_train))
print("Test score is:", model5.score(X_test,Y_test))

In [ ]:
res=[]
for i in Y_test_pred5:

    res.append(round(i))

Train with GA features

In [ ]:
model5.fit(x_train_GA, y_train_ga.values.ravel())
print("Train score is:", model5.score(x_train_GA, y_train_ga))

Train with CSF features

In [ ]:
model5.fit(x_train_cfs, y_train_cfs.values.ravel())
print("Train score is:", model5.score(x_train_cfs, y_train_cfs))

In [ ]:
cm = confusion_matrix(res, result_tot)

ConfusionMatrixDisplay(cm, model5.classes_).plot()

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(Y_test, Y_test_pred5)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(Y_test, Y_test_pred5)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(Y_test, Y_test_pred5)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(Y_test, Y_test_pred5)
print('F1 score: %f' % f1)